In [129]:
import numpy as np
import pandas as pd
from pathmgmt import pathmgmt as myPath
import os
from itertools import combinations
import random
import time

In [120]:
# Combine all alphas in the category
def get_stat(category):
    res = pd.DataFrame()
    for folderName in os.listdir(myPath.PLOT_DIR):
        if 'zz9999' not in folderName or not any(ele in folderName for ele in category):
            continue
        folder = myPath.PLOT_DIR/folderName/'statistics'
        for fileName in os.listdir(folder):
            if 'holding1days' not in fileName:
                continue
            file = folder/fileName
            df = pd.read_csv(file)
            df.index = [f'{folderName};{fileName[:-15]}']
            if res.empty:
                res = df
            else:
                res = pd.concat([res, df])
    return res

In [121]:
# combine daily pnl
def get_pnl(category):
    res = pd.DataFrame()
    for folderName in os.listdir(myPath.PLOT_DIR):
        if 'zz9999' not in folderName or not any(ele in folderName for ele in category):
            continue
        folder = myPath.PLOT_DIR/folderName/'PnL_results'
        for fileName in os.listdir(folder):
            file = folder/fileName
            if 'holding1days' not in fileName:
                continue
            df = pd.read_csv(file)
            df = df[['time', 'pnl']]
            df.rename(
                columns={'pnl': f'{folderName};{fileName[:-13]}'}, inplace=True)
            if res.empty:
                res = df
            else:
                res = res.merge(df, on='time', how='outer')
    res = res.set_index('time').sort_index()
    return res

In [133]:
def filter_good_alphas(category, good_alphas, number_to_be_selected, threshold=600):
    start = time.time()
    pnls = get_pnl(category)
    corr = pnls[good_alphas].corr()
    res = []
    num = number_to_be_selected
    good_candidates = list(((corr.mask(np.eye(len(corr), dtype=bool)).abs() < 0.6).sum() > num).index)
    random.shuffle(good_candidates)
    for cols in combinations(good_candidates, num):
        corr_small = corr.loc[cols, cols]
        if (~(corr_small.mask(np.eye(len(corr_small), dtype=bool)).abs() > 0.6).any()).sum() == num:
            res.append(corr_small)
        if time.time()-start > threshold:
            print(f'time out! exceed {threshold} seconds')
            break
    return res

In [140]:
def preprocess(category):
    stats = get_stat(category)
    print('------------Summary of Alphas------------')
    print(stats.describe())
    print('-----------------------------------------')
    good_alphas = list(stats.loc[(stats.IR > 2)].index)
    print(f'We are left with a total of {len(good_alphas)} alphas after the IR filter')
    return stats, good_alphas

In [136]:
def select_best_alpha(res, metric):
    max_IR = float('-inf')
    max_idx = -1
    for i in range(len(res)):
        total_IR = np.sum([(stats.loc[stats.index == name][metric]) for name in res[i].index])
        if total_IR > max_IR:
            max_IR = total_IR
            max_idx = i
    return max_idx

### Basic PV: Momentum and Reversion

In [141]:
pv_stats, good__alphas = preprocess(['ReV', 'MoM'])

------------Summary of Alphas------------
       Annualized Return  Annualized Excess Return         IR  IR long only  \
count          72.000000                 72.000000  72.000000     72.000000   
mean            0.128441                 -0.329299   1.514259     -1.784368   
std             0.791592                  0.533797   3.600627      2.143878   
min            -1.999030                 -2.225365  -7.818725     -8.813214   
25%             0.075087                 -0.214622   1.067800     -1.565124   
50%             0.191044                 -0.123925   2.413418     -0.994369   
75%             0.285351                 -0.087215   3.057507     -0.660842   
max             1.999030                 -0.023521   7.818725     -0.199495   

              IC  Max Drawdown  Max Drawdown long only  daily Turnover  
count  72.000000     72.000000               72.000000       72.000000  
mean    0.007975      0.655444                0.969447        0.398590  
std     0.013139      1.428

In [134]:
filtered__alphas = filter_good_alphas(['ReV', 'MoM'], good__alphas, 6, threshold=600)

time out! exceed 180 seconds


In [137]:
best_alpha_index = select_best_alpha(filtered__alphas, metric='IR long only')

In [84]:
filtered__alphas[best_alpha_index]

,alpha.YaoReV009-3days-a5-zz9999;20180101-20201231-Rank-Industry-holding1days,alpha.YaoReV005-20days-zz9999;20180101-20201231-None-Industry-holding1days,alpha.YaoMoM007-5days-zz9999;20180101-20201231-None-Industry-holding1days,alpha.YaoReV009-10days-a3-zz9999;20180101-20201231-Rank-Industry-holding1days,alpha.YaoReV001-60days-zz9999;20180101-20201231-None-Industry-holding1days,alpha.YaoMoM004-60days-zz9999;20180101-20201231-None-Industry-holding1days
alpha.YaoReV009-3days-a5-zz9999;20180101-20201231-Rank-Industry-holding1days,1.000000,0.321735,-0.060333,0.589686,0.038281,0.075535
alpha.YaoReV005-20days-zz9999;20180101-20201231-None-Industry-holding1days,0.321735,1.000000,-0.388371,0.195317,0.477801,-0.163930
alpha.YaoMoM007-5days-zz9999;20180101-20201231-None-Industry-holding1days,-0.060333,-0.388371,1.000000,-0.084678,-0.288954,0.483068
alpha.YaoReV009-10days-a3-zz9999;20180101-20201231-Rank-Industry-holding1days,0.589686,0.195317,-0.084678,1.000000,0.515029,-0.188020
alpha.YaoReV001-60days-zz9999;20180101-20201231-None-Industry-holding1days,0.038281,0.477801,-0.288954,0.515029,1.000000,-0.425162
alpha.YaoMoM004-60days-zz9999;20180101-20201231-None-Industry-holding1days,0.075535,-0.163930,0.483068,-0.188020,-0.425162,1.000000


In [89]:
pv_stats.loc[pv_stats.index.isin(filtered__alphas[best_alpha_index].index)].IR

alpha.YaoMoM004-60days-zz9999;20180101-20201231-None-Industry-holding1days       5.103285
alpha.YaoMoM007-5days-zz9999;20180101-20201231-None-Industry-holding1days        6.552738
alpha.YaoReV001-60days-zz9999;20180101-20201231-None-Industry-holding1days       2.451816
alpha.YaoReV005-20days-zz9999;20180101-20201231-None-Industry-holding1days       2.898292
alpha.YaoReV009-10days-a3-zz9999;20180101-20201231-Rank-Industry-holding1days    3.202771
alpha.YaoReV009-3days-a5-zz9999;20180101-20201231-Rank-Industry-holding1days     3.471707
Name: IR, dtype: float64

In [ ]:
pv_stats.loc[pv_stats.index.isin(filtered__alphas[best_alpha_index].index)].IC

### Technical

In [147]:
tec_stats, good__alphas_tec = preprocess(['Tec'])

------------Summary of Alphas------------
       Annualized Return  Annualized Excess Return          IR  IR long only  \
count         122.000000                122.000000  122.000000    122.000000   
mean            0.150919                 -0.117247    1.801836     -0.845064   
std             0.138424                  0.087986    1.470915      0.664449   
min            -0.289147                 -0.392629   -3.176756     -2.989204   
25%             0.085462                 -0.156682    1.211188     -1.069554   
50%             0.150392                 -0.112620    2.100646     -0.810342   
75%             0.234713                 -0.067510    2.705784     -0.474018   
max             0.512042                  0.053906    4.506186      0.619075   

               IC  Max Drawdown  Max Drawdown long only  daily Turnover  
count  122.000000    122.000000              122.000000      122.000000  
mean     0.009929      0.108414                0.400548        0.431105  
std      0.0082

In [111]:
filtered__alphas_tec = filter_good_alphas(['Tec'], good__alphas_tec, 6, threshold=600)

In [ ]:
best_alpha_index_tec = select_best_alpha(filtered__alphas_tec, metric='IR long only')

In [113]:
filtered__alphas_tec[best_alpha_index_tec]

,alpha.YaoTec003-3days-zz9999;20180101-20201231-None-Industry-holding1days,alpha.YaoTec011-10days-10days-zz9999;20180101-20201231-Rank-Industry-holding1days,alpha.YaoTec007-60days-zz9999;20180101-20201231-None-Industry-holding1days,alpha.YaoTec008-20days-zz9999;20180101-20201231-None-Industry-holding1days,alpha.YaoTec007-3days-zz9999;20180101-20201231-None-Industry-holding1days,alpha.YaoTec006-5days-zz9999;20180101-20201231-None-Industry-holding1days
alpha.YaoTec003-3days-zz9999;20180101-20201231-None-Industry-holding1days,1.000000,0.494141,0.316501,0.265346,0.249409,0.172228
alpha.YaoTec011-10days-10days-zz9999;20180101-20201231-Rank-Industry-holding1days,0.494141,1.000000,-0.012386,0.032304,-0.092294,0.514537
alpha.YaoTec007-60days-zz9999;20180101-20201231-None-Industry-holding1days,0.316501,-0.012386,1.000000,0.382625,0.595275,-0.017954
alpha.YaoTec008-20days-zz9999;20180101-20201231-None-Industry-holding1days,0.265346,0.032304,0.382625,1.000000,0.422812,0.218453
alpha.YaoTec007-3days-zz9999;20180101-20201231-None-Industry-holding1days,0.249409,-0.092294,0.595275,0.422812,1.000000,-0.022946
alpha.YaoTec006-5days-zz9999;20180101-20201231-None-Industry-holding1days,0.172228,0.514537,-0.017954,0.218453,-0.022946,1.000000


In [114]:
tec_stats.loc[tec_stats.index.isin(filtered__alphas_tec[best_alpha_index_tec].index)]

,Annualized Return,Annualized Excess Return,IR,IR long only,IC,Max Drawdown,Max Drawdown long only,daily Turnover
alpha.YaoTec003-3days-zz9999;20180101-20201231-None-Industry-holding1days,0.512042,0.047667,4.506186,0.527128,0.021574,0.070065,0.126021,0.571315
alpha.YaoTec006-5days-zz9999;20180101-20201231-None-Industry-holding1days,0.114543,-0.027970,2.274438,-0.283133,0.007211,0.034354,0.194278,0.875131
alpha.YaoTec007-3days-zz9999;20180101-20201231-None-Industry-holding1days,0.213971,-0.034304,2.885066,-0.308454,0.014931,0.062589,0.181945,0.831101
alpha.YaoTec007-60days-zz9999;20180101-20201231-None-Industry-holding1days,0.223065,0.035177,3.349317,0.350993,0.014482,0.050943,0.131480,0.188725
alpha.YaoTec008-20days-zz9999;20180101-20201231-None-Industry-holding1days,0.093473,-0.030465,2.521566,-0.287613,0.006980,0.026991,0.194620,0.445618
alpha.YaoTec011-10days-10days-zz9999;20180101-20201231-Rank-Industry-holding1days,0.186222,-0.103289,3.027162,-0.760397,0.010671,0.056118,0.336636,0.686290


In [149]:
tec_stats.loc[tec_stats.index.isin(filtered__alphas_tec[best_alpha_index_tec].index)].IR

alpha.YaoTec003-3days-zz9999;20180101-20201231-None-Industry-holding1days            4.506186
alpha.YaoTec006-5days-zz9999;20180101-20201231-None-Industry-holding1days            2.274438
alpha.YaoTec007-3days-zz9999;20180101-20201231-None-Industry-holding1days            2.885066
alpha.YaoTec007-60days-zz9999;20180101-20201231-None-Industry-holding1days           3.349317
alpha.YaoTec008-20days-zz9999;20180101-20201231-None-Industry-holding1days           2.521566
alpha.YaoTec011-10days-10days-zz9999;20180101-20201231-Rank-Industry-holding1days    3.027162
Name: IR, dtype: float64

In [150]:
tec_stats.loc[tec_stats.index.isin(filtered__alphas_tec[best_alpha_index_tec].index)].IC

alpha.YaoTec003-3days-zz9999;20180101-20201231-None-Industry-holding1days            0.021574
alpha.YaoTec006-5days-zz9999;20180101-20201231-None-Industry-holding1days            0.007211
alpha.YaoTec007-3days-zz9999;20180101-20201231-None-Industry-holding1days            0.014931
alpha.YaoTec007-60days-zz9999;20180101-20201231-None-Industry-holding1days           0.014482
alpha.YaoTec008-20days-zz9999;20180101-20201231-None-Industry-holding1days           0.006980
alpha.YaoTec011-10days-10days-zz9999;20180101-20201231-Rank-Industry-holding1days    0.010671
Name: IC, dtype: float64

### Volatility

In [151]:
vol_stats, good__alphas_vol = preprocess(['Vol'])

------------Summary of Alphas------------
       Annualized Return  Annualized Excess Return          IR  IR long only  \
count         117.000000                117.000000  115.000000    117.000000   
mean            0.280946                 -0.063536    2.794060     -0.464322   
std             0.382874                  0.078805    1.990407      0.631678   
min            -0.048368                 -0.221433   -1.050400     -1.522890   
25%             0.095442                 -0.114182    1.895514     -0.946132   
50%             0.202205                 -0.066664    2.646815     -0.549393   
75%             0.353471                 -0.005630    3.753681     -0.071817   
max             2.295026                  0.094528    9.161987      0.986050   

               IC  Max Drawdown  Max Drawdown long only  daily Turnover  
count  115.000000    117.000000              117.000000      117.000000  
mean     0.011663      0.078538                0.284673        0.389514  
std      0.0091

In [156]:
filtered__alphas_vol = filter_good_alphas(['Vol'], good__alphas_vol, 5, threshold=600)

time out! exceed 600 seconds


In [158]:
best_alpha_index_vol = select_best_alpha(filtered__alphas_vol, metric='IR long only')

In [159]:
filtered__alphas_vol[best_alpha_index_vol]

,alpha.YaoVol006-60days-zz9999;20180101-20201231-None-Industry-holding1days,alpha.YaoVol011-20days-60days-zz9999;20180101-20201231-Rank-Industry-holding1days,alpha.YaoVol002-20days-zz9999;20180101-20201231-None-Industry-holding1days,alpha.YaoVol005-10days-zz9999;20180101-20201231-None-Industry-holding1days,alpha.YaoVol007-20days-zz9999;20180101-20201231-Rank-Industry-holding1days
alpha.YaoVol006-60days-zz9999;20180101-20201231-None-Industry-holding1days,1.000000,0.149697,0.018299,-0.065876,0.026872
alpha.YaoVol011-20days-60days-zz9999;20180101-20201231-Rank-Industry-holding1days,0.149697,1.000000,0.248511,0.219394,0.502597
alpha.YaoVol002-20days-zz9999;20180101-20201231-None-Industry-holding1days,0.018299,0.248511,1.000000,0.315198,0.269630
alpha.YaoVol005-10days-zz9999;20180101-20201231-None-Industry-holding1days,-0.065876,0.219394,0.315198,1.000000,0.375395
alpha.YaoVol007-20days-zz9999;20180101-20201231-Rank-Industry-holding1days,0.026872,0.502597,0.269630,0.375395,1.000000


In [160]:
vol_stats.loc[vol_stats.index.isin(filtered__alphas_vol[best_alpha_index_vol].index)]

,Annualized Return,Annualized Excess Return,IR,IR long only,IC,Max Drawdown,Max Drawdown long only,daily Turnover
alpha.YaoVol002-20days-zz9999;20180101-20201231-None-Industry-holding1days,0.146045,-0.004083,2.956636,-0.038415,0.009854,0.062617,0.226621,0.270052
alpha.YaoVol005-10days-zz9999;20180101-20201231-None-Industry-holding1days,0.095928,-0.016116,2.803632,-0.168738,0.006330,0.033193,0.203754,0.655253
alpha.YaoVol006-60days-zz9999;20180101-20201231-None-Industry-holding1days,0.257544,0.062634,3.184895,0.731541,0.009846,0.079804,0.102883,0.047901
alpha.YaoVol007-20days-zz9999;20180101-20201231-Rank-Industry-holding1days,0.096142,-0.146240,2.810485,-1.083677,0.007097,0.025384,0.395617,0.305708
alpha.YaoVol011-20days-60days-zz9999;20180101-20201231-Rank-Industry-holding1days,0.123231,-0.162549,2.243600,-1.131034,0.009085,0.061336,0.443041,0.186171


In [161]:
vol_stats.loc[vol_stats.index.isin(filtered__alphas_vol[best_alpha_index_vol].index)].IR

alpha.YaoVol002-20days-zz9999;20180101-20201231-None-Industry-holding1days           2.956636
alpha.YaoVol005-10days-zz9999;20180101-20201231-None-Industry-holding1days           2.803632
alpha.YaoVol006-60days-zz9999;20180101-20201231-None-Industry-holding1days           3.184895
alpha.YaoVol007-20days-zz9999;20180101-20201231-Rank-Industry-holding1days           2.810485
alpha.YaoVol011-20days-60days-zz9999;20180101-20201231-Rank-Industry-holding1days    2.243600
Name: IR, dtype: float64

In [162]:
vol_stats.loc[vol_stats.index.isin(filtered__alphas_vol[best_alpha_index_vol].index)].IC

alpha.YaoVol002-20days-zz9999;20180101-20201231-None-Industry-holding1days           0.009854
alpha.YaoVol005-10days-zz9999;20180101-20201231-None-Industry-holding1days           0.006330
alpha.YaoVol006-60days-zz9999;20180101-20201231-None-Industry-holding1days           0.009846
alpha.YaoVol007-20days-zz9999;20180101-20201231-Rank-Industry-holding1days           0.007097
alpha.YaoVol011-20days-60days-zz9999;20180101-20201231-Rank-Industry-holding1days    0.009085
Name: IC, dtype: float64

### Liquidity

In [163]:
lqd_stats, good__alphas_lqd = preprocess(['Lqd'])

------------Summary of Alphas------------
       Annualized Return  Annualized Excess Return          IR  IR long only  \
count         199.000000                199.000000  199.000000    199.000000   
mean            0.235427                 -0.017616    3.010427     -0.139430   
std             0.232559                  0.087922    2.237120      0.767404   
min            -0.645902                 -0.819293   -3.980280     -4.448931   
25%             0.101580                 -0.044283    1.514155     -0.443767   
50%             0.213508                 -0.005114    3.455547     -0.048776   
75%             0.338185                  0.022624    4.685626      0.237136   
max             1.297202                  0.135350    7.926831      2.070061   

               IC  Max Drawdown  Max Drawdown long only  daily Turnover  
count  199.000000    199.000000              199.000000      199.000000  
mean     0.012233      0.099394                0.207035        0.647966  
std      0.0102

In [170]:
filtered__alphas_lqd = filter_good_alphas(['Lqd'], good__alphas_lqd, 4, threshold=600)

time out! exceed 600 seconds


In [171]:
best_alpha_index_lqd = select_best_alpha(filtered__alphas_lqd, metric='IR long only')

In [172]:
filtered__alphas_lqd[best_alpha_index_lqd]

,alpha.YaoLqd001-10days-zz9999;20180101-20201231-Rank-Industry-holding1days,alpha.YaoLqd006-20days-a05-zz9999;20180101-20201231-None-Industry-holding1days,alpha.YaoLqd009-3days-zz9999;20180101-20201231-None-Industry-holding1days,alpha.YaoLqd015-3days-zz9999;20180101-20201231-None-Industry-holding1days
alpha.YaoLqd001-10days-zz9999;20180101-20201231-Rank-Industry-holding1days,1.000000,0.005018,0.317935,0.321903
alpha.YaoLqd006-20days-a05-zz9999;20180101-20201231-None-Industry-holding1days,0.005018,1.000000,0.208870,-0.026981
alpha.YaoLqd009-3days-zz9999;20180101-20201231-None-Industry-holding1days,0.317935,0.208870,1.000000,0.084279
alpha.YaoLqd015-3days-zz9999;20180101-20201231-None-Industry-holding1days,0.321903,-0.026981,0.084279,1.000000


In [173]:
lqd_stats.loc[lqd_stats.index.isin(filtered__alphas_lqd[best_alpha_index_lqd].index)]

,Annualized Return,Annualized Excess Return,IR,IR long only,IC,Max Drawdown,Max Drawdown long only,daily Turnover
alpha.YaoLqd001-10days-zz9999;20180101-20201231-Rank-Industry-holding1days,0.273586,0.027703,4.674443,0.260224,0.019527,0.040908,0.179060,0.948476
alpha.YaoLqd006-20days-a05-zz9999;20180101-20201231-None-Industry-holding1days,0.136401,0.017940,2.283723,0.246748,0.005942,0.069148,0.111675,0.083508
alpha.YaoLqd009-3days-zz9999;20180101-20201231-None-Industry-holding1days,0.612138,-0.036209,5.505706,-0.373726,0.032402,0.105052,0.164897,0.409657
alpha.YaoLqd015-3days-zz9999;20180101-20201231-None-Industry-holding1days,0.604578,0.013780,4.291100,0.132795,0.019803,0.111674,0.154003,0.363014


In [174]:
lqd_stats.loc[lqd_stats.index.isin(filtered__alphas_lqd[best_alpha_index_lqd].index)].IR

alpha.YaoLqd001-10days-zz9999;20180101-20201231-Rank-Industry-holding1days        4.674443
alpha.YaoLqd006-20days-a05-zz9999;20180101-20201231-None-Industry-holding1days    2.283723
alpha.YaoLqd009-3days-zz9999;20180101-20201231-None-Industry-holding1days         5.505706
alpha.YaoLqd015-3days-zz9999;20180101-20201231-None-Industry-holding1days         4.291100
Name: IR, dtype: float64

In [175]:
lqd_stats.loc[lqd_stats.index.isin(filtered__alphas_lqd[best_alpha_index_lqd].index)].IC

alpha.YaoLqd001-10days-zz9999;20180101-20201231-Rank-Industry-holding1days        0.019527
alpha.YaoLqd006-20days-a05-zz9999;20180101-20201231-None-Industry-holding1days    0.005942
alpha.YaoLqd009-3days-zz9999;20180101-20201231-None-Industry-holding1days         0.032402
alpha.YaoLqd015-3days-zz9999;20180101-20201231-None-Industry-holding1days         0.019803
Name: IC, dtype: float64

### Money Flow

In [176]:
mfl_stats, good__alphas_mfl = preprocess(['Mfl'])

------------Summary of Alphas------------
       Annualized Return  Annualized Excess Return         IR  IR long only  \
count          95.000000                 95.000000  95.000000     95.000000   
mean            0.220414                 -0.018151   2.694962     -0.108886   
std             0.134799                  0.107830   1.447540      0.886896   
min             0.001160                 -0.371735   0.017717     -2.616625   
25%             0.107835                 -0.090653   1.498822     -0.868143   
50%             0.228251                 -0.000096   2.822629     -0.000885   
75%             0.312137                  0.066793   3.782142      0.668207   
max             0.570503                  0.120719   6.109713      1.078943   

              IC  Max Drawdown  Max Drawdown long only  daily Turnover  
count  95.000000     95.000000               95.000000       95.000000  
mean    0.012251      0.082424                0.251354        0.301569  
std     0.008089      0.027

In [181]:
filtered__alphas_mfl = filter_good_alphas(['Mfl'], good__alphas_mfl, 5, threshold=600)

time out! exceed 600 seconds


In [183]:
best_alpha_index_mfl = select_best_alpha(filtered__alphas_mfl, metric='IR long only')

In [184]:
filtered__alphas_mfl[best_alpha_index_mfl]

,alpha.YaoMfl003-5days-zz9999;20180101-20201231-None-Industry-holding1days,alpha.YaoMfl005-60days-zz9999;20180101-20201231-None-Industry-holding1days,alpha.YaoMfl002-20days-zz9999;20180101-20201231-None-Industry-holding1days,alpha.YaoMfl008-3days-zz9999;20180101-20201231-None-Industry-holding1days,alpha.YaoMfl016-60days-zz9999;20180101-20201231-None-Industry-holding1days
alpha.YaoMfl003-5days-zz9999;20180101-20201231-None-Industry-holding1days,1.000000,-0.201018,-0.261529,0.360498,0.447223
alpha.YaoMfl005-60days-zz9999;20180101-20201231-None-Industry-holding1days,-0.201018,1.000000,0.529322,0.431356,-0.021251
alpha.YaoMfl002-20days-zz9999;20180101-20201231-None-Industry-holding1days,-0.261529,0.529322,1.000000,0.362366,0.190797
alpha.YaoMfl008-3days-zz9999;20180101-20201231-None-Industry-holding1days,0.360498,0.431356,0.362366,1.000000,0.438970
alpha.YaoMfl016-60days-zz9999;20180101-20201231-None-Industry-holding1days,0.447223,-0.021251,0.190797,0.438970,1.000000


In [185]:
mfl_stats.loc[mfl_stats.index.isin(filtered__alphas_mfl[best_alpha_index_mfl].index)]

,Annualized Return,Annualized Excess Return,IR,IR long only,IC,Max Drawdown,Max Drawdown long only,daily Turnover
alpha.YaoMfl002-20days-zz9999;20180101-20201231-None-Industry-holding1days,0.254276,0.088350,2.612018,0.795580,0.013814,0.081590,0.139172,0.052527
alpha.YaoMfl003-5days-zz9999;20180101-20201231-None-Industry-holding1days,0.129431,-0.106864,2.007744,-0.981482,0.009709,0.052983,0.336844,0.590823
alpha.YaoMfl005-60days-zz9999;20180101-20201231-None-Industry-holding1days,0.194683,0.047482,3.713988,0.520036,0.011335,0.039608,0.109260,0.024948
alpha.YaoMfl008-3days-zz9999;20180101-20201231-None-Industry-holding1days,0.379357,0.058089,5.589981,0.530141,0.025604,0.055486,0.152506,0.265938
alpha.YaoMfl016-60days-zz9999;20180101-20201231-None-Industry-holding1days,0.215803,-0.118701,3.238267,-0.973660,0.009616,0.057643,0.320615,0.154759


In [186]:
mfl_stats.loc[mfl_stats.index.isin(filtered__alphas_mfl[best_alpha_index_mfl].index)].IR

alpha.YaoMfl002-20days-zz9999;20180101-20201231-None-Industry-holding1days    2.612018
alpha.YaoMfl003-5days-zz9999;20180101-20201231-None-Industry-holding1days     2.007744
alpha.YaoMfl005-60days-zz9999;20180101-20201231-None-Industry-holding1days    3.713988
alpha.YaoMfl008-3days-zz9999;20180101-20201231-None-Industry-holding1days     5.589981
alpha.YaoMfl016-60days-zz9999;20180101-20201231-None-Industry-holding1days    3.238267
Name: IR, dtype: float64

In [187]:
mfl_stats.loc[mfl_stats.index.isin(filtered__alphas_mfl[best_alpha_index_mfl].index)].IC

alpha.YaoMfl002-20days-zz9999;20180101-20201231-None-Industry-holding1days    0.013814
alpha.YaoMfl003-5days-zz9999;20180101-20201231-None-Industry-holding1days     0.009709
alpha.YaoMfl005-60days-zz9999;20180101-20201231-None-Industry-holding1days    0.011335
alpha.YaoMfl008-3days-zz9999;20180101-20201231-None-Industry-holding1days     0.025604
alpha.YaoMfl016-60days-zz9999;20180101-20201231-None-Industry-holding1days    0.009616
Name: IC, dtype: float64

### Level 2 

we combine the combined alphas from the 5 categories

In [188]:
lvl2_stats, good__alphas_lvl2 = preprocess(['combined'])

------------Summary of Alphas------------
       Annualized Return  Annualized Excess Return         IR  IR long only  \
count          10.000000                 10.000000  10.000000     10.000000   
mean            0.660571                  0.072868   6.619874      0.758281   
std             0.408969                  0.038761   2.054467      0.409108   
min             0.298545                  0.032626   4.707861      0.352547   
25%             0.425931                  0.053346   5.432340      0.513752   
50%             0.479224                  0.061391   5.612756      0.629458   
75%             0.725541                  0.065188   7.101858      0.703627   
max             1.508279                  0.147125  10.655995      1.546464   

              IC  Max Drawdown  Max Drawdown long only  daily Turnover  
count  10.000000     10.000000               10.000000       10.000000  
mean    0.027896      0.056824                0.127916        0.458747  
std     0.006902      0.014

In [196]:
pnls = get_pnl(['combined'])
corr = pnls[good__alphas_lvl2].corr()

In [197]:
corr

,alpha.combined-Lqd-equalweight-zz9999;20180101-20201231-None-Industry-holding1days,alpha.combined-Lqd-irweight-zz9999;20180101-20201231-None-Industry-holding1days,alpha.combined-Mfl-equalweight-zz9999;20180101-20201231-None-Industry-holding1days,alpha.combined-Mfl-irweight-zz9999;20180101-20201231-None-Industry-holding1days,alpha.combined-PV-equalweight-zz9999;20180101-20201231-None-Industry-holding1days,alpha.combined-PV-irweight-zz9999;20180101-20201231-None-Industry-holding1days,alpha.combined-Tec-equalweight-zz9999;20180101-20201231-None-Industry-holding1days,alpha.combined-Tec-irweight-zz9999;20180101-20201231-None-Industry-holding1days,alpha.combined-Vol-equalweight-zz9999;20180101-20201231-None-Industry-holding1days,alpha.combined-Vol-irweight-zz9999;20180101-20201231-None-Industry-holding1days
alpha.combined-Lqd-equalweight-zz9999;20180101-20201231-None-Industry-holding1days,1.000000,0.989044,0.608864,0.640865,0.356769,0.060335,0.711504,0.737370,0.724931,0.712277
alpha.combined-Lqd-irweight-zz9999;20180101-20201231-None-Industry-holding1days,0.989044,1.000000,0.604622,0.638054,0.327356,0.051283,0.690132,0.706699,0.691240,0.675475
alpha.combined-Mfl-equalweight-zz9999;20180101-20201231-None-Industry-holding1days,0.608864,0.604622,1.000000,0.987770,0.232253,-0.067722,0.674214,0.716631,0.785121,0.777047
alpha.combined-Mfl-irweight-zz9999;20180101-20201231-None-Industry-holding1days,0.640865,0.638054,0.987770,1.000000,0.248561,-0.038241,0.663887,0.710633,0.789414,0.782611
alpha.combined-PV-equalweight-zz9999;20180101-20201231-None-Industry-holding1days,0.356769,0.327356,0.232253,0.248561,1.000000,0.906544,0.306365,0.324557,0.301317,0.302373
alpha.combined-PV-irweight-zz9999;20180101-20201231-None-Industry-holding1days,0.060335,0.051283,-0.067722,-0.038241,0.906544,1.000000,-0.040660,-0.034342,-0.037195,-0.025904
alpha.combined-Tec-equalweight-zz9999;20180101-20201231-None-Industry-holding1days,0.711504,0.690132,0.674214,0.663887,0.306365,-0.040660,1.000000,0.989935,0.728367,0.709446
alpha.combined-Tec-irweight-zz9999;20180101-20201231-None-Industry-holding1days,0.737370,0.706699,0.716631,0.710633,0.324557,-0.034342,0.989935,1.000000,0.770849,0.754096
alpha.combined-Vol-equalweight-zz9999;20180101-20201231-None-Industry-holding1days,0.724931,0.691240,0.785121,0.789414,0.301317,-0.037195,0.728367,0.770849,1.000000,0.995559
alpha.combined-Vol-irweight-zz9999;20180101-20201231-None-Industry-holding1days,0.712277,0.675475,0.777047,0.782611,0.302373,-0.025904,0.709446,0.754096,0.995559,1.000000


In [198]:
lvl2_stats

,Annualized Return,Annualized Excess Return,IR,IR long only,IC,Max Drawdown,Max Drawdown long only,daily Turnover
alpha.combined-Lqd-equalweight-zz9999;20180101-20201231-None-Industry-holding1days,0.684158,0.064803,7.040704,0.709619,0.034176,0.065714,0.121186,0.554829
alpha.combined-Lqd-irweight-zz9999;20180101-20201231-None-Industry-holding1days,0.739335,0.061173,7.122243,0.621295,0.036570,0.072314,0.132325,0.614784
alpha.combined-Mfl-equalweight-zz9999;20180101-20201231-None-Industry-holding1days,0.422698,0.050737,5.456058,0.487064,0.025448,0.069691,0.124603,0.277571
alpha.combined-Mfl-irweight-zz9999;20180101-20201231-None-Industry-holding1days,0.435630,0.061419,5.769453,0.593816,0.026812,0.061222,0.125551,0.227651
alpha.combined-PV-equalweight-zz9999;20180101-20201231-None-Industry-holding1days,1.251780,0.140223,10.655995,1.546464,0.036132,0.027137,0.103742,0.368651
alpha.combined-PV-irweight-zz9999;20180101-20201231-None-Industry-holding1days,1.508279,0.147125,9.719494,1.467068,0.032304,0.036888,0.121512,0.485504
alpha.combined-Tec-equalweight-zz9999;20180101-20201231-None-Industry-holding1days,0.454128,0.032626,5.453688,0.352547,0.025632,0.059529,0.156429,0.676036
alpha.combined-Tec-irweight-zz9999;20180101-20201231-None-Industry-holding1days,0.504319,0.043899,5.425224,0.481663,0.026739,0.067974,0.148625,0.640771
alpha.combined-Vol-equalweight-zz9999;20180101-20201231-None-Industry-holding1days,0.298545,0.061363,4.707861,0.637621,0.017540,0.055365,0.124180,0.372326
alpha.combined-Vol-irweight-zz9999;20180101-20201231-None-Industry-holding1days,0.306839,0.065317,4.848018,0.685653,0.017605,0.052407,0.121010,0.369352


In [199]:
lvl2_stats.IR

alpha.combined-Lqd-equalweight-zz9999;20180101-20201231-None-Industry-holding1days     7.040704
alpha.combined-Lqd-irweight-zz9999;20180101-20201231-None-Industry-holding1days        7.122243
alpha.combined-Mfl-equalweight-zz9999;20180101-20201231-None-Industry-holding1days     5.456058
alpha.combined-Mfl-irweight-zz9999;20180101-20201231-None-Industry-holding1days        5.769453
alpha.combined-PV-equalweight-zz9999;20180101-20201231-None-Industry-holding1days     10.655995
alpha.combined-PV-irweight-zz9999;20180101-20201231-None-Industry-holding1days         9.719494
alpha.combined-Tec-equalweight-zz9999;20180101-20201231-None-Industry-holding1days     5.453688
alpha.combined-Tec-irweight-zz9999;20180101-20201231-None-Industry-holding1days        5.425224
alpha.combined-Vol-equalweight-zz9999;20180101-20201231-None-Industry-holding1days     4.707861
alpha.combined-Vol-irweight-zz9999;20180101-20201231-None-Industry-holding1days        4.848018
Name: IR, dtype: float64

In [200]:
lvl2_stats.IC

alpha.combined-Lqd-equalweight-zz9999;20180101-20201231-None-Industry-holding1days    0.034176
alpha.combined-Lqd-irweight-zz9999;20180101-20201231-None-Industry-holding1days       0.036570
alpha.combined-Mfl-equalweight-zz9999;20180101-20201231-None-Industry-holding1days    0.025448
alpha.combined-Mfl-irweight-zz9999;20180101-20201231-None-Industry-holding1days       0.026812
alpha.combined-PV-equalweight-zz9999;20180101-20201231-None-Industry-holding1days     0.036132
alpha.combined-PV-irweight-zz9999;20180101-20201231-None-Industry-holding1days        0.032304
alpha.combined-Tec-equalweight-zz9999;20180101-20201231-None-Industry-holding1days    0.025632
alpha.combined-Tec-irweight-zz9999;20180101-20201231-None-Industry-holding1days       0.026739
alpha.combined-Vol-equalweight-zz9999;20180101-20201231-None-Industry-holding1days    0.017540
alpha.combined-Vol-irweight-zz9999;20180101-20201231-None-Industry-holding1days       0.017605
Name: IC, dtype: float64